# Dikkat Mekanizmaları ve Transformerlar

Tekrarlayan ağların (recurrent networks) en büyük dezavantajlarından biri, bir dizideki tüm kelimelerin sonuca aynı etkiyi yapmasıdır. Bu durum, Adlandırılmış Varlık Tanıma (Named Entity Recognition) ve Makine Çevirisi gibi dizi-dizi görevlerinde standart LSTM encoder-decoder modelleriyle optimal olmayan bir performansa yol açar. Gerçekte, giriş dizisindeki belirli kelimeler genellikle ardışık çıktılar üzerinde diğerlerinden daha fazla etkiye sahiptir.

Makine çevirisi gibi bir dizi-dizi modelini düşünün. Bu model, iki tekrarlayan ağ tarafından uygulanır: biri (**encoder**) giriş dizisini gizli bir duruma sıkıştırır, diğeri (**decoder**) bu gizli durumu çevrilmiş bir sonuca açar. Bu yaklaşımın sorunu, ağın son durumunun bir cümlenin başlangıcını hatırlamakta zorlanmasıdır, bu da uzun cümlelerde modelin kalitesinin düşmesine neden olur.

**Dikkat Mekanizmaları**, her bir giriş vektörünün RNN'nin her bir çıktı tahmini üzerindeki bağlamsal etkisini ağırlıklandırmanın bir yolunu sağlar. Bu, giriş RNN'nin ara durumları ile çıkış RNN arasında kısayollar oluşturarak uygulanır. Bu şekilde, çıktı sembolü $y_t$ oluşturulurken, farklı ağırlık katsayıları $\alpha_{t,i}$ ile tüm giriş gizli durumlarını $h_i$ dikkate alırız.

![Eklenecek bir dikkat katmanı ile encoder/decoder modelini gösteren görsel](../../../../../translated_images/tr/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau ve ark., 2015](https://arxiv.org/pdf/1409.0473.pdf) çalışmasından alınan ve [bu blog yazısından](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) alıntılanan eklemeli dikkat mekanizmalı encoder-decoder modeli]*

Dikkat matrisi $\{\alpha_{i,j}\}$, belirli giriş kelimelerinin çıktı dizisindeki bir kelimenin oluşturulmasında oynadığı rol derecesini temsil eder. Aşağıda böyle bir matrisin örneği verilmiştir:

![Bahdanau - arviz.org'dan alınan RNNsearch-50 tarafından bulunan örnek bir hizalamayı gösteren görsel](../../../../../translated_images/tr/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau ve ark., 2015](https://arxiv.org/pdf/1409.0473.pdf) çalışmasından alınan (Şekil 3)]*

Dikkat mekanizmaları, Doğal Dil İşleme alanında mevcut veya yakın zamanda mevcut olan en iyi performansın büyük bir kısmından sorumludur. Ancak dikkat eklemek, model parametrelerinin sayısını büyük ölçüde artırır ve bu da RNN'lerde ölçekleme sorunlarına yol açar. RNN'lerin ölçeklenmesindeki temel kısıtlama, modellerin tekrarlayan doğasının eğitimde toplu işlem ve paralelleştirmeyi zorlaştırmasıdır. Bir RNN'de bir dizinin her bir öğesi sıralı bir şekilde işlenmelidir, bu da kolayca paralelleştirilemeyeceği anlamına gelir.

Dikkat mekanizmalarının benimsenmesi ve bu kısıtlama, bugün BERT'ten OpenGPT3'e kadar bildiğimiz ve kullandığımız en iyi performanslı Transformer Modellerinin oluşturulmasına yol açtı.

## Transformer Modelleri

Her bir önceki tahminin bağlamını bir sonraki değerlendirme adımına iletmek yerine, **transformer modelleri** bir metin penceresi içinde verilen bir girişin bağlamını yakalamak için **konumsal kodlamalar** ve dikkat mekanizmasını kullanır. Aşağıdaki görsel, konumsal kodlamaların dikkat ile bir pencere içinde bağlamı nasıl yakalayabileceğini göstermektedir.

![Transformer modellerinde değerlendirmelerin nasıl yapıldığını gösteren animasyonlu GIF.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Her bir giriş pozisyonu bağımsız olarak her bir çıkış pozisyonuna eşlendiğinden, transformerlar RNN'lere göre daha iyi paralelleştirilebilir, bu da çok daha büyük ve daha ifade gücü yüksek dil modellerini mümkün kılar. Her bir dikkat başlığı, kelimeler arasındaki farklı ilişkileri öğrenmek için kullanılabilir ve bu da Doğal Dil İşleme görevlerini iyileştirir.

**BERT** (Bidirectional Encoder Representations from Transformers), *BERT-base* için 12 katman ve *BERT-large* için 24 katman içeren çok büyük bir çok katmanlı transformer ağıdır. Model, büyük bir metin veri kümesi (WikiPedia + kitaplar) üzerinde denetimsiz eğitim (bir cümledeki maskelenmiş kelimeleri tahmin etme) kullanılarak önce önceden eğitilir. Önceden eğitim sırasında model, önemli bir dil anlayışı seviyesini emer ve bu daha sonra diğer veri kümeleriyle ince ayar yapılarak kullanılabilir. Bu sürece **transfer öğrenme** denir.

![http://jalammar.github.io/illustrated-bert/ adresinden alınan görsel](../../../../../translated_images/tr/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Transformer mimarilerinin BERT, DistilBERT, BigBird, OpenGPT3 ve daha fazlası gibi birçok varyasyonu vardır ve bunlar ince ayar yapılabilir. [HuggingFace paketi](https://github.com/huggingface/) PyTorch ile bu mimarilerin birçoğunu eğitmek için bir depo sağlar.

## BERT'i Metin Sınıflandırması için Kullanma

Şimdi önceden eğitilmiş BERT modelini geleneksel görevimizi çözmek için nasıl kullanabileceğimize bakalım: dizi sınıflandırması. Orijinal AG News veri setimizi sınıflandıracağız.

Öncelikle HuggingFace kütüphanesini ve veri setimizi yükleyelim:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Önceden eğitilmiş BERT modelini kullanacağımız için, belirli bir tokenizer kullanmamız gerekecek. İlk olarak, önceden eğitilmiş BERT modeliyle ilişkili bir tokenizer yükleyeceğiz.

HuggingFace kütüphanesi, yalnızca isimlerini `from_pretrained` fonksiyonlarına argüman olarak belirterek kullanabileceğiniz önceden eğitilmiş modellerin bir deposunu içerir. Model için gerekli olan tüm ikili dosyalar otomatik olarak indirilecektir.

Ancak, bazı durumlarda kendi modellerinizi yüklemeniz gerekebilir. Bu durumda, tokenizer için parametreler, model parametrelerini içeren `config.json` dosyası, ikili ağırlıklar ve diğer ilgili dosyaları içeren dizini belirtebilirsiniz.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` nesnesi, metni doğrudan kodlamak için kullanılabilecek `encode` işlevini içerir:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Sonra, eğitim sırasında verilere erişmek için kullanacağımız yineleyiciler oluşturalım. Çünkü BERT kendi kodlama fonksiyonunu kullanır, daha önce tanımladığımız `padify`'a benzer bir doldurma fonksiyonu tanımlamamız gerekecek:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Bizim durumumuzda, `bert-base-uncased` adlı önceden eğitilmiş BERT modelini kullanacağız. Modeli `BertForSequenceClassfication` paketi kullanarak yükleyelim. Bu, modelimizin sınıflandırma için gerekli mimariye, son sınıflandırıcı dahil, zaten sahip olmasını sağlar. Son sınıflandırıcının ağırlıklarının başlatılmadığını ve modelin ön eğitim gerektireceğini belirten bir uyarı mesajı göreceksiniz - bu tamamen normaldir, çünkü tam olarak yapacağımız şey budur!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Artık eğitime başlamaya hazırız! BERT zaten önceden eğitildiği için, başlangıç ağırlıklarını bozmamak adına oldukça küçük bir öğrenme oranıyla başlamak istiyoruz.

Tüm ağır işi `BertForSequenceClassification` modeli yapıyor. Modeli eğitim verileri üzerinde çağırdığımızda, giriş minibatch'i için hem kayıp (loss) hem de ağ çıktısını döndürüyor. Kayıp, parametre optimizasyonu için kullanılıyor (`loss.backward()` geri yayılımı gerçekleştirir) ve `out`, elde edilen etiketleri `argmax` kullanarak hesaplanan `labs` ile beklenen `labels` değerlerini karşılaştırarak eğitim doğruluğunu hesaplamak için kullanılıyor.

Süreci kontrol edebilmek için, birkaç yineleme boyunca kayıp ve doğruluğu biriktiriyor ve her `report_freq` eğitim döngüsünde bunları yazdırıyoruz.

Bu eğitim muhtemelen oldukça uzun sürecek, bu yüzden yineleme sayısını sınırlıyoruz.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


BERT sınıflandırmasının (özellikle yineleme sayısını artırıp yeterince beklerseniz) oldukça iyi bir doğruluk sağladığını görebilirsiniz! Bunun nedeni, BERT'in dilin yapısını zaten oldukça iyi anlaması ve bizim yalnızca son sınıflandırıcıyı ince ayar yapmamızın gerekmesidir. Ancak, BERT büyük bir model olduğu için tüm eğitim süreci uzun sürer ve ciddi bir hesaplama gücü gerektirir! (GPU ve tercihen birden fazla).

> **Not:** Örneğimizde, önceden eğitilmiş en küçük BERT modellerinden birini kullanıyoruz. Daha iyi sonuçlar verme olasılığı yüksek olan daha büyük modeller de mevcuttur.


## Model performansını değerlendirme

Şimdi modelimizin test veri seti üzerindeki performansını değerlendirebiliriz. Değerlendirme döngüsü, eğitim döngüsüne oldukça benzerdir, ancak `model.eval()` çağrısı yaparak modeli değerlendirme moduna geçirmeyi unutmamalıyız.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Özet

Bu bölümde, **transformers** kütüphanesinden önceden eğitilmiş bir dil modelini alıp, metin sınıflandırma görevimize uyarlamanın ne kadar kolay olduğunu gördük. Benzer şekilde, BERT modelleri varlık çıkarımı, soru yanıtlama ve diğer NLP görevleri için de kullanılabilir.

Transformer modelleri, NLP alanında güncel en iyi durumu temsil eder ve çoğu durumda özel NLP çözümleri uygularken denemeye başlayacağınız ilk çözüm olmalıdır. Ancak, bu modülde ele alınan tekrarlayan sinir ağlarının temel prensiplerini anlamak, gelişmiş sinir modelleri oluşturmak istiyorsanız son derece önemlidir.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluğu sağlamak için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
